In [ ]:
# CREATED: 21-DEC-2024
# LAST EDIT: 22-DEC-2024
# AUTHOR: DUANE RINEHART, MBA (duane.rinehart@gmail.com)

#SAFE TO DELETE
'''
GOALS:
1) ANALYZE HISTORICAL STOCK INFORMATION, CURRENT OPTIONS PREMIUMS (COVERED CALLS) TO
CALCULATE EXPECTED VALUE OF OPTIONS CONTRACTS (AND COMPARE TO CURRENT PRICES)

2) CALCULATE ANNUALIZED RETURN ON [COVERED CALLS] OPTIONS CONTRACTS


INPUTS:
1) EXCEL FILE OF STOCKS OF INTEREST (WHERE >= 100 SHARES ARE OWNED)
2) API TO FINANCIAL INFORMAITON SERVICE (E.G. YAHOO FINANCE) FOR HISTORICAL STOCK INFORMATION

OUTPUTS:
2) EXCEL DOCUMENT CONTAINING ANALYSIS OF STOCK LIST (CALCS)

INTERMEDIATE DATA STORE:
SQL LITE

FILE & EXCEL LOCATIONS DEFINED IN constants.json.  See setup.ipynb for creation


'''

import pandas as pd
import numpy as np
import json
import ast
from data_access import YahooAPI
from dashboard import populate_db, db_table_stats
from data_access import HistData
from pathlib import Path
import sqlite3 


def load_app_constants():
    # Load constants from constants.json
    constants_file_path = 'constants.json'

    try:
        with open(constants_file_path, 'r') as file:
            content = file.read()
            # Adjust the content to be compatible with Python syntax
            content = content.replace("constants = {", "{").replace("};", "}")
            # Use ast.literal_eval to safely evaluate the string as a Python dictionary
            constants = ast.literal_eval(content)
            return constants
    except FileNotFoundError:
        print(f"File {constants_file_path} not found.  Did you run setup.ipynb?")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def db_last_update(dict_constants):
    db_server = dict_constants.get('HOST_DB_SERVER', 'sqlite')
    db_name = dict_constants.get('HOST_DB_NAME', 'finance.db')

    results = db_table_stats(db_server, db_name, 'history')
    print('TICKER', 'START_DATE', 'END_DATE', sep='\t')
    print('------', '----------', '--------', sep='\t')
    [print(row[0], row[1], row[2], sep='\t') for row in results]


def db_update(dict_constants):
    '''
    populate_db assume table to be populated is 'history', consistent with db_schema.sql
    '''
    print("Updating database...")
    db_server = dict_constants.get('HOST_DB_SERVER', 'sqlite')
    db_name = dict_constants.get('HOST_DB_NAME', 'finance.db')
    input_excel_file_path = dict_constants.get('STOCK_EXCEL_INPUT_FILE_PATH', '')
    input_excel_file_name = dict_constants.get('STOCK_EXCEL_INPUT_FILE_NAME', 'stock_blotter.xlsx')
    input_excel_stocks = Path(input_excel_file_path, input_excel_file_name)
    
    # Read the Excel file into a DataFrame (from 'inputs' worksheet)
    df = pd.read_excel(input_excel_stocks, sheet_name='inputs')
    symbols = df['SYMBOL'].tolist() #EXTRACT JUST SYMBOLS

    populate_db(db_server, db_name, 'history', symbols, '5d')


def create_output_analysis(dict_constants):
    db_server = dict_constants.get('HOST_DB_SERVER', 'sqlite')
    db_name = dict_constants.get('HOST_DB_NAME', 'finance.db')
    input_excel_file_path = dict_constants.get('STOCK_EXCEL_INPUT_FILE_PATH', '')
    input_excel_file_name = dict_constants.get('STOCK_EXCEL_INPUT_FILE_NAME', 'stock_blotter.xlsx')
    input_excel_stocks = Path(input_excel_file_path, input_excel_file_name)
    output_excel_file_path = Path(dict_constants.get('STOCK_EXCEL_OUTPUT_FILE_PATH', 'output'))
    output_excel_file_name = dict_constants.get('STOCK_EXCEL_OUTPUT_FILE_NAME', 'analysis_output.xlsx')
    output_excel_analysis = Path(output_excel_file_path, output_excel_file_name)

    # Read the Excel file into a DataFrame (from 'inputs' worksheet)
    df_input = pd.read_excel(input_excel_stocks, sheet_name='inputs')
    symbols = df_input['SYMBOL'].tolist() #EXTRACT JUST SYMBOLS
    sql_where = "('" + "', '".join(symbols) + "')"
    conn = sqlite3.connect(db_name)  # Replace with your DB connection details
    sql = f'SELECT * FROM history WHERE symbol IN {sql_where}'
    df = pd.read_sql(sql, conn) 
    conn.close()

    #CALCS FOR ANLALYSIS OUTPUT
    # df_analysis = pd.concat([df_input['SYMBOL'], 'PRICE_EST'], axis=1)
    df_analysis = df_input[['SYMBOL']].copy()

    output_excel_file_path.mkdir(parents=True, exist_ok=True)
    try:
        with pd.ExcelWriter(output_excel_analysis, mode='w', engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name='historical')
            df_analysis.to_excel(writer, index=False, sheet_name='analysis')
        print(f"Data exported successfully to '{output_excel_analysis}'")
    except PermissionError as e:
        print(f"PermissionError: {e}. Please check file permissions and ensure the file is not open in another program.")
    except FileNotFoundError as e:
        print(f"FileNotFoundError: {e}. The specified directory or file does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}. Please check the file path and ensure all necessary libraries are installed.")
        


def calc_monthly_standard_deviation(stock):
    # Example data
    data = pd.read_csv('stock_data.csv')
    data['Returns'] = data['Close'].pct_change()
    std_dev = data['Returns'].std()
    monthly_std_dev = std_dev * np.sqrt(21)  # Approx. 21 trading days per month
    print(monthly_std_dev)


'''
constants = {
    "stock_symbol": "AAPL",
    "strike_price": 150,
    "option_premium": 2.5,
    "risk_free_rate": 0.05,
    "time_to_expiration_days": 30,
    "volatility_estimate": 0.2
}
'''
def display_menu():
    '''ENSURE CHOICE OPTIONS EXIST FOR EACH ITEM'''
    print('1 = UPDATE DATABASE')
    print('2 = SHOW LAST DATABASE INFO')
    print('3 = ANALYZE OPTIONS')
    print('M = DISPLAY MENU')
    print('X = EXIT')


def main():
    print('WELCOME TO OPTIONS ANALYSIS TRACKER')
    print('------------------------------------')
    dict_constants = load_app_constants()
    display_menu()

    while True:
        choice = input('ACTION: ')
        if choice == '1':
            db_update(dict_constants)
        if choice == '2':
            db_last_update(dict_constants)
        elif choice == '3':
            print('ANALYZING OPTIONS...')
            create_output_analysis(dict_constants)
        elif choice.upper() == 'M':
            display_menu()
        elif choice.upper() == 'X':
            print('GOODBYE')
            break
        else:
            print('INVALID CHOICE')


if __name__ == "__main__":
    main()

In [26]:
import pandas as pd
import numpy as np
import json
import ast
from data_access import YahooAPI
from dashboard import populate_db, db_table_stats
from data_access import HistData
from pathlib import Path
import sqlite3 
from datetime import datetime

def load_app_constants():
    # Load constants from constants.json
    constants_file_path = 'constants.json'

    try:
        with open(constants_file_path, 'r') as file:
            content = file.read()
            # Adjust the content to be compatible with Python syntax
            content = content.replace("constants = {", "{").replace("};", "}")
            # Use ast.literal_eval to safely evaluate the string as a Python dictionary
            constants = ast.literal_eval(content)
            return constants
    except FileNotFoundError:
        print(f"File {constants_file_path} not found.  Did you run setup.ipynb?")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def create_output_analysis(dict_constants):
    db_server = dict_constants.get('HOST_DB_SERVER', 'sqlite')
    db_name = dict_constants.get('HOST_DB_NAME', 'finance.db')
    input_excel_file_path = dict_constants.get('STOCK_EXCEL_INPUT_FILE_PATH', '')
    input_excel_file_name = dict_constants.get('STOCK_EXCEL_INPUT_FILE_NAME', 'stock_blotter.xlsx')
    input_excel_stocks = Path(input_excel_file_path, input_excel_file_name)
    output_excel_file_path = Path(dict_constants.get('STOCK_EXCEL_OUTPUT_FILE_PATH', 'output'))
    output_excel_file_name = dict_constants.get('STOCK_EXCEL_OUTPUT_FILE_NAME', 'analysis_output.xlsx')
    output_excel_analysis = Path(output_excel_file_path, output_excel_file_name)

    # Read the Excel file into a DataFrame (from 'inputs' worksheet)
    df_input = pd.read_excel(input_excel_stocks, sheet_name='inputs')
    symbols = df_input['SYMBOL'].tolist() #EXTRACT JUST SYMBOLS
    sql_where = "('" + "', '".join(symbols) + "')"
    conn = sqlite3.connect(db_name)  # Replace with your DB connection details
    sql = f'SELECT * FROM history WHERE symbol IN {sql_where}'
    df = pd.read_sql(sql, conn) 
    conn.close()

    #CALCS FOR ANLALYSIS OUTPUT
    # df_analysis = pd.concat([df_input['SYMBOL'], 'PRICE_EST'], axis=1)
    df_analysis = df_input[['SYMBOL']].copy()
    return df, df_analysis

dict_constants = load_app_constants()
df, df_analysis = create_output_analysis(dict_constants)


In [25]:
df

,date,symbol,open,close,adj_close,high,low,volume
0,2024-12-16,MSFT,447.269989,451.589996,None,452.179993,445.279999,23598800
1,2024-12-17,MSFT,451.010010,454.459991,None,455.290009,449.570007,22733500
2,2024-12-18,MSFT,451.320007,437.390015,None,452.649994,437.019989,24444500
3,2024-12-19,MSFT,441.619995,437.029999,None,443.179993,436.320007,22963700
4,2024-12-20,MSFT,433.109985,436.600006,None,443.739990,428.630005,64235200
...,...,...,...,...,...,...,...,...
751,2024-09-18,F,10.783485,10.832814,None,11.109062,10.773619,55017300
752,2024-09-19,F,11.069597,10.773619,None,11.158391,10.724289,52436400
753,2024-09-20,F,10.714423,10.734156,None,10.734156,10.546702,68809100
754,2024-12-23,F,9.870000,9.900000,None,9.920000,9.750000,50646600


In [30]:
# df_analysis
df['date'] = pd.to_datetime(df['date'])

symbol = 'F'

# Filter the DataFrame for individual symbol
individual_stock_df = df[df['symbol'] == symbol]
#print(individual_stock_df)
# recent_month = individual_stock_df['date'].dt.month.max()
# recent_data = individual_stock_df[individual_stock_df['date'].dt.month == recent_month]

current_month = datetime.now().month
current_year = datetime.now().year

# Filter DataFrame for the current month and year
recent_data = individual_stock_df[
    (individual_stock_df['date'].dt.month == current_month) & 
    (individual_stock_df['date'].dt.year == current_year)
]
print(recent_data)
# Calculate the standard deviation of the 'close' column
std_dev = recent_data['close'].std()
print(std_dev)

          date symbol   open  close adj_close   high    low    volume
10  2024-12-16      F  10.07   9.99      None  10.12   9.88  96264400
11  2024-12-17      F   9.96   9.97      None  10.04   9.88  76208600
12  2024-12-18      F   9.94   9.69      None  10.09   9.66  84199500
13  2024-12-19      F   9.79   9.74      None   9.95   9.70  81659000
14  2024-12-20      F   9.70   9.88      None   9.96   9.68  87566200
182 2024-12-02      F  11.22  10.98      None  11.25  10.96  50034400
183 2024-12-03      F  10.99  10.82      None  11.01  10.82  43904300
184 2024-12-04      F  10.85  10.74      None  10.87  10.59  66780700
185 2024-12-05      F  10.64  10.44      None  10.69  10.39  68162000
186 2024-12-06      F  10.51  10.51      None  10.60  10.42  56227000
187 2024-12-09      F  10.56  10.60      None  10.80  10.53  53230100
188 2024-12-10      F  10.72  10.56      None  10.74  10.55  45084300
189 2024-12-11      F  10.61  10.41      None  10.61  10.28  60487100
190 2024-12-12      

In [17]:
recent_data

,date,symbol,open,close,adj_close,high,low,volume
10,2024-12-16,F,10.070000,9.990000,None,10.120000,9.880000,96264400
11,2024-12-17,F,9.960000,9.970000,None,10.040000,9.880000,76208600
12,2024-12-18,F,9.940000,9.690000,None,10.090000,9.660000,84199500
13,2024-12-19,F,9.790000,9.740000,None,9.950000,9.700000,81659000
14,2024-12-20,F,9.700000,9.880000,None,9.960000,9.680000,87566200
182,2024-12-02,F,11.220000,10.980000,None,11.250000,10.960000,50034400
183,2024-12-03,F,10.990000,10.820000,None,11.010000,10.820000,43904300
184,2024-12-04,F,10.850000,10.740000,None,10.870000,10.590000,66780700
185,2024-12-05,F,10.640000,10.440000,None,10.690000,10.390000,68162000
186,2024-12-06,F,10.510000,10.510000,None,10.600000,10.420000,56227000


In [28]:


# Remove invalid dates if any
recent_data = recent_data.dropna(subset=['date'])

# Sort chronologically (oldest to newest)
df_sorted = recent_data.sort_values(by='date', ascending=True).reset_index(drop=True)

In [29]:
df_sorted

,date,symbol,open,close,adj_close,high,low,volume
0,2024-12-02,F,11.22,10.98,None,11.25,10.96,50034400
1,2024-12-03,F,10.99,10.82,None,11.01,10.82,43904300
2,2024-12-04,F,10.85,10.74,None,10.87,10.59,66780700
3,2024-12-05,F,10.64,10.44,None,10.69,10.39,68162000
4,2024-12-06,F,10.51,10.51,None,10.60,10.42,56227000
5,2024-12-09,F,10.56,10.60,None,10.80,10.53,53230100
6,2024-12-10,F,10.72,10.56,None,10.74,10.55,45084300
7,2024-12-11,F,10.61,10.41,None,10.61,10.28,60487100
8,2024-12-12,F,10.38,10.39,None,10.51,10.35,38432600
9,2024-12-13,F,10.40,10.39,None,10.44,10.32,40560700


In [15]:
# Ensure 'date' is in datetime format
recent_data['date'] = pd.to_datetime(recent_data['date'], errors='coerce')

# Remove invalid dates if any
recent_data = recent_data.dropna(subset=['date'])

# Sort chronologically (oldest to newest)
df_sorted = recent_data.sort_values(by='date', ascending=True).reset_index(drop=True)

# Display sorted DataFrame
print(df_sorted)

         date symbol       open      close adj_close       high        low   
0  2023-12-26      F  11.579544  11.635619      None  11.682349  11.560852  \
1  2023-12-27      F  11.616929  11.579545      None  11.635620  11.430010   
2  2023-12-28      F  11.570198  11.532815      None  11.616928  11.476739   
3  2023-12-29      F  11.542161  11.392626      None  11.579544  11.364588   
4  2024-12-02      F  11.220000  10.980000      None  11.250000  10.960000   
5  2024-12-03      F  10.990000  10.820000      None  11.010000  10.820000   
6  2024-12-04      F  10.850000  10.740000      None  10.870000  10.590000   
7  2024-12-05      F  10.640000  10.440000      None  10.690000  10.390000   
8  2024-12-06      F  10.510000  10.510000      None  10.600000  10.420000   
9  2024-12-09      F  10.560000  10.600000      None  10.800000  10.530000   
10 2024-12-10      F  10.720000  10.560000      None  10.740000  10.550000   
11 2024-12-11      F  10.610000  10.410000      None  10.610000 